In [1]:
from src.recommenders import ImprovedRecommender
from sklearn.decomposition import TruncatedSVD, PCA, IncrementalPCA, KernelPCA, SparsePCA
import pandas as pd
import numpy as np
from os import cpu_count
from multiprocessing import Pool
from itertools import combinations
import pickle

In [2]:
qual_eval_folder = './evaluation'
items_path = "./data/games.pkl"
data_path = "./data/interactions_splits_"
reviews_path = "./data/reviews.parquet"

In [3]:
def average_results(results):
    average_dict = {}
    for key in results[0]:
        key_list = [result[key] for result in results]
        average_dict[key] = sum(key_list)/len(key_list)
    return average_dict


In [9]:
split_results = []
for split in [data_path + str(i) for i in range(3)]:
    rec = ImprovedRecommender(items_path, train_path=f"{split}_train.parquet", test_path=f"{split}_test.parquet", val_path=f"{split}_val.parquet", reviews_path=reviews_path, sparse=True, tfidf='smooth', normalize=True)
    rec.generate_recommendations(read_max=1000)
    split_results.append(rec.evaluate(k=10, val=True))

with open('./evaluation/results.pickle', 'wb') as handle:
    pickle.dump(average_results(split_results), handle, protocol=pickle.HIGHEST_PROTOCOL)
print(average_results(split_results))

1000it [00:01, 619.28it/s]
1000it [00:02, 391.61it/s]
1000it [00:01, 608.11it/s]
1000it [00:02, 394.59it/s]
1000it [00:01, 602.65it/s]
1000it [00:02, 391.95it/s]


{'HR@10': 0.28833333333333333, 'nDCG@10': 0.08141360603426297, 'recall@10': 0.10015726969805705, 'ideal_recall@10': 0.935365173648156, 'nRecall@10': 0.10452896825396825}


Let's test with different item representations!

In [12]:
information = ['specs', 'genres', 'early_access', 'publisher', 'developer']
def test_representation(columns, split):
    rec = ImprovedRecommender(items_path, train_path=f"{split}_train.parquet", test_path=f"{split}_test.parquet", val_path=f"{split}_val.parquet", reviews_path=reviews_path, sparse=True, tfidf='smooth', normalize=True, columns=columns, weighting_scheme={})
    rec.generate_recommendations(read_max=5000)
    return rec.evaluate(k=10)

results = []
representations = [map(list, combinations(information, r)) for r in range(1, len(information)+1)]
representations = [rep+['tags'] for sublist in representations for rep in sublist]
for columns in representations:
    split_results = []
    for split in [data_path + str(i) for i in range(3)]:
        split_results.append(test_representation(columns, split))
    results.append(average_results(split_results))
    print(columns, results[-1])

with open('./evaluation/representations.pickle', 'wb') as handle:
    pickle.dump(representations, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('./evaluation/representations_results.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

5000it [00:00, 8446.30it/s]
5000it [00:10, 472.60it/s]
5000it [00:00, 8531.39it/s]
5000it [00:10, 474.53it/s]
5000it [00:00, 7721.89it/s]
5000it [00:10, 467.30it/s]


['specs', 'tags'] {'HR@10': 0.1902, 'nDCG@10': 0.051539207908820474, 'recall@10': 0.06717211479669648, 'ideal_recall@10': 0.9429034701783022, 'nRecall@10': 0.06939227513227514}


5000it [00:00, 8516.16it/s]
5000it [00:09, 518.66it/s]
5000it [00:00, 8410.25it/s]
5000it [00:09, 521.11it/s]
5000it [00:00, 8665.30it/s]
5000it [00:09, 524.44it/s]


['genres', 'tags'] {'HR@10': 0.19013333333333335, 'nDCG@10': 0.04837447216211648, 'recall@10': 0.06477183772303335, 'ideal_recall@10': 0.9429034701783022, 'nRecall@10': 0.06733669312169312}


5000it [00:00, 8675.92it/s]
5000it [00:09, 512.81it/s]
5000it [00:00, 8757.40it/s]
5000it [00:09, 507.66it/s]
5000it [00:00, 8127.73it/s]
5000it [00:10, 495.57it/s]


...

['specs', 'early_access', 'publisher', 'developer', 'tags'] {'HR@10': 0.26366666666666666, 'nDCG@10': 0.0792388269982653, 'recall@10': 0.09852182418310478, 'ideal_recall@10': 0.9429034701783022, 'nRecall@10': 0.1018154232804233}


5000it [00:02, 2157.93it/s]
5000it [00:13, 381.56it/s]
5000it [00:02, 2210.30it/s]
5000it [00:12, 390.10it/s]
5000it [00:02, 2169.08it/s]
5000it [00:13, 382.99it/s]


['genres', 'early_access', 'publisher', 'developer', 'tags'] {'HR@10': 0.259, 'nDCG@10': 0.07588120370769746, 'recall@10': 0.09660555468863653, 'ideal_recall@10': 0.9429034701783022, 'nRecall@10': 0.09995698412698413}


5000it [00:02, 2121.00it/s]
5000it [00:14, 349.93it/s]
5000it [00:02, 2077.72it/s]
5000it [00:14, 349.85it/s]
5000it [00:02, 2104.36it/s]
5000it [00:14, 344.27it/s]


['specs', 'genres', 'early_access', 'publisher', 'developer', 'tags'] {'HR@10': 0.26286666666666664, 'nDCG@10': 0.0790052078761835, 'recall@10': 0.09815592627909164, 'ideal_recall@10': 0.9429034701783022, 'nRecall@10': 0.10144746031746033}


Lets test different weighting schemes! :-)

In [4]:
weighting_schemes = [{'playtime': playtime, 'sentiment': sentiment, 'reviews': reviews} for playtime in [True, False] for sentiment in ['rating', 'n_reviews', 'mixed', False] for reviews in [True, False]]
def test_weighting_scheme(rec, weighting_scheme):
    rec.set_weighting_scheme(weighting_scheme)
    rec.generate_recommendations(read_max=5000, silence=True)
    return rec.evaluate(k=10)

results = []
recs = [ImprovedRecommender(items_path, train_path=f"{split}_train.parquet", test_path=f"{split}_test.parquet", val_path=f"{split}_val.parquet", reviews_path=reviews_path, sparse=True, tfidf='smooth', normalize=True) for split in [data_path + str(i) for i in range(3)]]
for weighting_scheme in weighting_schemes:
    split_results = []
    for rec in recs:
        split_results.append(test_weighting_scheme(rec, weighting_scheme))
    results.append(average_results(split_results))
    print(weighting_scheme, results[-1])
    
with open('./evaluation/weighting_schemes.pickle', 'wb') as handle:
    pickle.dump(weighting_schemes, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('./evaluation/weighting_schemes_results.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

{'playtime': True, 'sentiment': 'rating', 'reviews': True} {'HR@10': 0.17933333333333334, 'nDCG@10': 0.0475884260251058, 'recall@10': 0.058465005786153414, 'ideal_recall@10': 0.9429034701783022, 'nRecall@10': 0.06174928571428571}
{'playtime': True, 'sentiment': 'rating', 'reviews': False} {'HR@10': 0.17926666666666669, 'nDCG@10': 0.04753410167307553, 'recall@10': 0.05851466147328083, 'ideal_recall@10': 0.9429034701783022, 'nRecall@10': 0.061776984126984125}
{'playtime': True, 'sentiment': 'n_reviews', 'reviews': True} {'HR@10': 0.1774, 'nDCG@10': 0.04757885825547136, 'recall@10': 0.05916118510745074, 'ideal_recall@10': 0.9429034701783022, 'nRecall@10': 0.06237457671957672}
{'playtime': True, 'sentiment': 'n_reviews', 'reviews': False} {'HR@10': 0.1769333333333333, 'nDCG@10': 0.04765522202738313, 'recall@10': 0.059140959475475895, 'ideal_recall@10': 0.9429034701783022, 'nRecall@10': 0.0623126455026455}
{'playtime': True, 'sentiment': 'mixed', 'reviews': True} {'HR@10': 0.175933333333333

c:\Users\Sam\Documents\GitHub\AI-Project\src\recommenders.py:348: RuntimeWarning: Both playtime and sentiment in weighting were set to 'False', using playtime.
  warnings.warn("Both playtime and sentiment in weighting were set to 'False', using playtime.", RuntimeWarning)
c:\Users\Sam\Documents\GitHub\AI-Project\src\recommenders.py:348: RuntimeWarning: Both playtime and sentiment in weighting were set to 'False', using playtime.
  warnings.warn("Both playtime and sentiment in weighting were set to 'False', using playtime.", RuntimeWarning)
c:\Users\Sam\Documents\GitHub\AI-Project\src\recommenders.py:348: RuntimeWarning: Both playtime and sentiment in weighting were set to 'False', using playtime.
  warnings.warn("Both playtime and sentiment in weighting were set to 'False', using playtime.", RuntimeWarning)


{'playtime': False, 'sentiment': False, 'reviews': True} {'HR@10': 0.264, 'nDCG@10': 0.07926497169109241, 'recall@10': 0.09857171213828686, 'ideal_recall@10': 0.9429034701783022, 'nRecall@10': 0.10187431216931216}


c:\Users\Sam\Documents\GitHub\AI-Project\src\recommenders.py:348: RuntimeWarning: Both playtime and sentiment in weighting were set to 'False', using playtime.
  warnings.warn("Both playtime and sentiment in weighting were set to 'False', using playtime.", RuntimeWarning)
c:\Users\Sam\Documents\GitHub\AI-Project\src\recommenders.py:348: RuntimeWarning: Both playtime and sentiment in weighting were set to 'False', using playtime.
  warnings.warn("Both playtime and sentiment in weighting were set to 'False', using playtime.", RuntimeWarning)
c:\Users\Sam\Documents\GitHub\AI-Project\src\recommenders.py:348: RuntimeWarning: Both playtime and sentiment in weighting were set to 'False', using playtime.
  warnings.warn("Both playtime and sentiment in weighting were set to 'False', using playtime.", RuntimeWarning)


{'playtime': False, 'sentiment': False, 'reviews': False} {'HR@10': 0.26366666666666666, 'nDCG@10': 0.0792388269982653, 'recall@10': 0.09852182418310478, 'ideal_recall@10': 0.9429034701783022, 'nRecall@10': 0.1018154232804233}


Finally, let's check out dimensionality reduction

In [4]:
dim_reds = [TruncatedSVD(n_components=300, random_state=500), PCA(n_components=300, random_state=500, whiten=False), IncrementalPCA(n_components=300, whiten=False), KernelPCA(n_components=300)]
labels = ["TruncatedSVD(n_components=300)", "PCA(n_components=300)", "IncrementalPCA(n_components=300)", "KernelPCA(n_components=300)"]
# use_data = ['specs', 'genres', 'tags', 'early_access', 'publisher', 'developer']
results = []
for dim_red in dim_reds:
    split_results = []
    for split in [data_path + str(i) for i in range(3)]:
        rec = ImprovedRecommender(items_path, train_path=f"{split}_train.parquet", test_path=f"{split}_test.parquet", val_path=f"{split}_val.parquet", reviews_path=reviews_path, sparse=False, tfidf='smooth', normalize=True, dim_red=dim_red)
        rec.generate_recommendations(read_max=5000)
        split_results.append(rec.evaluate(k=10))
    results.append(average_results(split_results))
    
with open('./evaluation/dim_reduction.pickle', 'wb') as handle:
    pickle.dump(labels, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('./evaluation/dim_reduction_results.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

5000it [00:00, 40143.87it/s]
5000it [02:00, 41.33it/s]
5000it [00:00, 34598.76it/s]
5000it [02:10, 38.31it/s]
5000it [00:00, 36323.63it/s]
5000it [02:01, 41.21it/s]
5000it [00:00, 32679.66it/s]
5000it [01:52, 44.61it/s]
5000it [00:00, 35696.63it/s]
5000it [02:06, 39.65it/s]
5000it [00:00, 33774.15it/s]
5000it [01:52, 44.53it/s]
5000it [00:00, 35954.40it/s]
5000it [02:01, 41.25it/s]
5000it [00:00, 30298.05it/s]
5000it [02:01, 41.30it/s]
5000it [00:00, 29749.96it/s]
5000it [02:09, 38.74it/s]
5000it [00:00, 26316.14it/s]
5000it [02:09, 38.50it/s]
5000it [00:00, 25895.40it/s]
5000it [02:05, 39.77it/s]
5000it [00:00, 23364.64it/s]
5000it [02:18, 36.20it/s]


In [ ]:
dim_red = SparsePCA(n_components=300, n_jobs=8)
rec = ImprovedRecommender(items_path, train_path=f"{data_path}_train.parquet", test_path=f"{data_path}_test.parquet", val_path=f"{data_path}_val.parquet", reviews_path=reviews_path, sparse=False, tfidf='smooth', normalize=True, dim_red=dim_red)
rec.generate_recommendations(read_max=1000) # read_max=1000
rec.evaluate(k=10)